In [10]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram,fcluster
import matplotlib.pyplot as plt

from obj  import Trajectory


In [11]:
class Coordinates:
    """
    Represents a point with x and y coordinates.

    Args:
        coordinateList (list): A list containing x and y coordinates.

    Attributes:
        x (float): The x-coordinate.
        y (float): The y-coordinate.
    """
    def __init__(self, coordinateList: list):
        self.x = float(coordinateList[0])
        self.y = float(coordinateList[1])

    def withinFilter(self, tar_cor: 'Coordinates', window_size: int) -> bool:
        """
        Checks if the current coordinate is within a given window around the target coordinate.

        Args:
            tar_cor (Coordinates): The target coordinate to compare with.
            window_size (float): The size of the window.

        Returns:
            bool: True if the current coordinate is within the window around the target coordinate, False otherwise.
        """
        
        if not isinstance(tar_cor, Coordinates):
            raise TypeError("tar_cor must be an instance of Coordinates.")
        
        if not isinstance(window_size, int):
            raise TypeError("window_size must be a float.")
            
        return (tar_cor.x <= self.x + window_size/2) and \
               (tar_cor.x >= self.x - window_size/2) and \
               (tar_cor.y <= self.y + window_size/2) and \
               (tar_cor.y >= self.y - window_size/2)

class Trajectory:
    """
    Represents a trajectory with a unique ID, type, and a list of coordinates.

    Args:
        entity (dict): A dictionary representing the trajectory entity.

    Attributes:
        id (str): The unique identifier of the trajectory.
        type (str): The type of the trajectory.
        traj (list): A list of Coordinates objects representing the trajectory path.
    """
    def __init__(self, entity: dict):
        self.id = entity['object_id']
        self.type = entity['object_type']
        self.traj = [Coordinates(cor.split(',')) for cor in entity['coordinates'].split()]

    def similarity(self, target_traj: 'Trajectory', window_size: int) -> float:
        """
        Calculates the similarity score of the two trajectories's direction between based on their coordinates.

        Args:
            target_traj (Trajectory): The target trajectory to compare with.
            window_size (float): The size of the window for coordinate comparison.

        Returns:

            float: The similarity score between the two trajectories. The score is within the range of 0 to 1,
                   where 0 indicates no similarity and 1 indicates maximum similarity.
        """
        
        
        if not isinstance(target_traj, Trajectory):
            raise TypeError("tar_cor must be an instance of Coordinates.")
        
        if not isinstance(window_size, int):
            raise TypeError("window_size must be a float.")
            
        score = 0
        for cor in self.traj:
            for tar in target_traj.traj:
                if cor.withinFilter(tar, window_size):
                    score += 1
                    break
        for cor in target_traj.traj:
            for tar in self.traj:
                if cor.withinFilter(tar, window_size):
                    score += 1
                    break
        return score / (len(self.traj) + len(target_traj.traj))
    def trajExport(self):
        """
        Extracts x and y coordinates from the trajectory.

        Returns:
            tuple: A tuple containing two lists. The first list contains x-coordinates and the second list contains y-coordinates.
        """
        x_coords = [cor.x for cor in self.traj]
        y_coords = [cor.y for cor in self.traj]
        return x_coords, y_coords

In [25]:
# Define hyper-parameters
windowSize = 200  # The window size for trajectory comparison
distance_threshold = 0.8  # The threshold for merge cluster 
sampleSize = 1000  # The number of trajectories to process

# Read and parse trajectory data from the JSON file
with open(r'trajectories.json') as user_file:
    file_contents = user_file.read()

parsed_json = json.loads(file_contents)

# Create a list of Trajectory instances from the parsed JSON data
parsed_trajectory = [Trajectory(entity) for entity in parsed_json[:sampleSize]]


In [ ]:
def generateDistanceMap(parsed_trajectory):
    """
    Generates a similarity-based distance map for a list of trajectories.

    Args:
        parsed_trajectory (list): A list of Trajectory instances.

    Returns:
        numpy.ndarray: A NumPy array containing the similarity-based distance map.
    """
    print(f'There are {len(parsed_trajectory)} trajectories in the dataset')

    # Initialize a list to store distance maps
    distance_map_list = []

    # Calculate the similarity-based distance map
    for entity in parsed_trajectory:
        # Initialize an array to store distances from the current entity
        disArray = []

        if len(disArray) % 10 == 9:
            print(f'{len(disArray)} of {len(parsed_trajectory)} trajectories has been processed')

        # Calculate similarity with all other entities
        for entity2 in parsed_trajectory:
            # Calculate the similarity between the two entities using the similarity method
            similarity = 1 - entity.similarity(entity2, windowSize)

            # Append the similarity-based distance to the array
            disArray.append(similarity)

        # Append the distances for the current entity to the distance map list
        distance_map_list.append(disArray)

    print(f'All trajectories have been processed')

    # Convert the distance map list to a NumPy array
    return np.asarray(distance_map_list, dtype=np.float32)

# Generate the distance map
distance_map = generateDistanceMap(parsed_trajectory)


There are 839 trajectories in the dataset


In [ ]:
# Perform hierarchical clustering
hierarchical_clusters = linkage(distance_map, method='single')  # Calculate hierarchical linkage
max_distance = distance_threshold * 2  # Maximum distance for flat clustering

# Flat clustering using the 'fcluster' function
clusters = fcluster(hierarchical_clusters, t=max_distance, criterion='distance')

# Create a dictionary to store cluster memberships
cluster_members = {}

# Assign data points to clusters
for idx, cluster_id in enumerate(clusters):
    if cluster_id in cluster_members:
        cluster_members[cluster_id].append(idx)
    else:
        cluster_members[cluster_id] = [idx]


In [ ]:
#colors for visualization
cluster_colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF',
                  '#00FFFF', '#FF8000', '#8000FF', '#0080FF', '#80FF00',
                  '#800000', '#008000', '#000080', '#808000', '#800080',
                  '#008080', '#808080', '#FFC0CB', '#FFD700', '#FFA500',
                  '#800000', '#ADFF2F', '#20B2AA', '#00BFFF', '#7B68EE',
                  '#FA8072', '#87CEFA', '#CD5C5C', '#F08080', '#40E0D0']

In [ ]:

numMostCommonGroupToDisplay=30
# Count the size of each cluster
cluster_sizes = Counter(clusters)
# Find the 10 largest clusters
largest_clusters = cluster_sizes.most_common(numMostCommonGroupToDisplay)
largest_clusters


    
big_cluster={len(big_cluster):cluster_members[cluster] for cluster, size in largest_clusters}

In [ ]:



for cluster_id, members in big_cluster.items():
    for mem in members:
        traj=parsed_trajectory[mem]
        x_coords, y_coords = traj.trajExport()
        plt.plot(x_coords, y_coords, color=cluster_colors[cluster_id])

# Add labels and title
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f'Most {numMostCommonGroupToDisplay} common route')
plt.legend()

# Display the plot
plt.show()

In [ ]:



for traj in parsed_trajectory:
    x_coords, y_coords = traj.trajExport()
    plt.plot(x_coords, y_coords, color='black')


for cluster_id, members in big_cluster.items():
    for mem in members:
        traj=parsed_trajectory[mem]
        x_coords, y_coords = traj.trajExport()
        plt.plot(x_coords, y_coords, color=cluster_colors[cluster_id])
# Add labels and title
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Scatter Plot of Points')
plt.legend()

# Display the plot
plt.show()